## 4. Implement search problem for Blocks World (CRN: 020-365, last digit 5)

Stack blocks on top of each other to achieve the goal state. Only one block on top of table or top of stack can be moved at a time provided nothing is on its own top.

In [1]:
from collections import deque


class BlocksWorld:
    def __init__(
        self,
        initial_state={"A": "C", "B": None, "C": None},
        goal_state={"A": None, "B": "A", "C": "B"},
    ):
        """
        Initialize open queue with initial state, goal state and closed set for visited nodes.
        In the state (as a dict), the keys represent block names and values represent
        names of blocks they are on top of. Value of `None` denotes being on top of table.
        """
        self.open = deque(
            [(None, initial_state)]
        )  # node pair is ordered pair of (parent, child)
        self.goal_state = goal_state
        self.closed = []

    def goal_test(self, state):
        """
        Check to see if blocks are arranged according to goal state.
        """
        for block in state.keys():
            if not state[block] == self.goal_state[block]:
                return False
        return True

    def successor(self, state):
        """
        Generate successor states based on current state and production rules.
        """
        free_blocks = [b for b in state.keys() if b not in state.values()]
        table_top_blocks = [b for b in free_blocks if state[b] is None]
        stack_top_blocks = [b for b in free_blocks if state[b] is not None]

        succ = []

        # Move stack top blocks to top of other stacks or to table
        for stb in stack_top_blocks:
            # Move to top of other stacks
            for a_stb in stack_top_blocks:
                if stb == a_stb:
                    continue
                next_state = state.copy()
                next_state.update({stb: a_stb})
                succ.append(next_state)
            # Move to table
            next_state = state.copy()
            next_state.update({stb: None})
            succ.append(next_state)

        # Move table top blocks to top of other free blocks
        for ttb in table_top_blocks:
            for fb in free_blocks:
                next_state = state.copy()
                next_state.update({ttb: fb})
                succ.append(next_state)

        return succ

    def get_pair_child(self, node_pair_iter):
        """
        Returns list of only children from iterable containing (parent, child) node pairs.
        """
        return [pair[1] for pair in node_pair_iter]

    def bfs(self):
        """
        Breadth First Search using open as queue.
        """
        while self.open:
            # Dequeue, add to closed set and check if current node is goal
            node_pair = self.open.popleft()
            self.closed.append(node_pair)
            _, node = node_pair
            if self.goal_test(node):
                return node
            # If current node is not goal, generate successors and add to open queue
            self.open += [
                (node, s)
                for s in self.successor(node)
                if s not in self.get_pair_child(self.open)
                and s not in self.get_pair_child(self.closed)
            ]
        # Return None if goal not found
        return None

    def generate_path(self):
        """
        Generate the path from initial state to solution/goal state.
        """
        path = []
        node = self.goal_state
        while node:
            path.append(str(node))
            for parent, child in self.closed:
                if node == child:
                    node = parent
                    break
        path.reverse()
        print("Solution path:", end=" ")
        print(" -> ".join(path))

    def run(self):
        """
        Driver method.
        """
        goal_node = self.bfs()
        if goal_node:
            self.generate_path()
            print(f"Found the goal state {goal_node}")
        else:
            print("Did not find goal state")


bw = BlocksWorld()
bw.run()

Solution path: {'A': 'C', 'B': None, 'C': None} -> {'A': None, 'B': None, 'C': None} -> {'A': None, 'B': 'A', 'C': None} -> {'A': None, 'B': 'A', 'C': 'B'}
Found the goal state {'A': None, 'B': 'A', 'C': 'B'}
